# Example of feature extraction from Convolutional Neural Networks (CNNs) for image classification

In order to run this example more quickly in Google Colab, you must activate the GPU, by choosing Runtime -> Change runtime type -> GPU

See: https://discuss.pytorch.org/t/how-to-extract-features-of-an-image-from-a-trained-model/119/3

Load the matplotlib library and set the graphics to be plotted within the Jupyter notebook

Also load the PyTorch library, and fix the random seed to ensure the reproducibility of the results

Finally, import some standard classifiers from the `sklearn` library

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np

Establish the 10 classes that are available in the CIFAR10 dataset, see https://www.cs.toronto.edu/~kriz/cifar.html

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

Download the training data for the CIFAR10 dataset, upzip it to the virtual 
folder ../data-unversioned/p1ch6/ and store it as the `cifar10` variable

In [3]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Extracting ../data-unversioned/p1ch6/cifar-10-python.tar.gz to ../data-unversioned/p1ch6/


Download the validation data for the CIFAR10 dataset, upzip it to the virtual 
folder ../data-unversioned/p1ch6/ and store it as the `cifar10_val` variable

In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


Select the classes that the CNN will classify, namely the airplane (index 0) and bird (index 2) classes

Then extract a subset of the full training CIFAR10 dataset which contains the training samples of those two classes into the `cifar2` variable, and a subset of the full validation CIFAR10 dataset with the validation samples of these two classes into the `cifar2_val` variable

In [6]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img.to(device=device), label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img.to(device=device), label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Load the pretrained AlexNet model, and define the `AlexNetConv4` model which contains all the layers of the AlexNet up to the conv4 layer

In [7]:
original_model = models.alexnet(pretrained=True).to(device=device)

print(original_model)

class AlexNetConv4(nn.Module):
            def __init__(self):
                super(AlexNetConv4, self).__init__()
                self.features = nn.Sequential(
                    # stop at conv4
                    *list(original_model.features.children())[:-3]
                )
            def forward(self, x):
                x = self.features(x)
                return x

model = AlexNetConv4().to(device=device)

print(model)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)


Load the first image of the training set, process it with the neural model, and find out the number of extracted features

In [8]:
img, _ = cifar2[0]
output = model(img.unsqueeze(0))
num_features = list(output.flatten().shape)[0]
num_features


256

Create a data loader for the training set and another one for the validation set

In [9]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)  # <1>
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

Extract the features for the training set

In [10]:
X_train = np.zeros((0,num_features))
y_train = np.zeros((0))
for imgs, labels in train_loader:  # <3>
            outputs = model(imgs)  # <4>
            X_train = np.concatenate((X_train,outputs.detach().cpu().numpy().squeeze()))
            y_train = np.concatenate((y_train, labels))


Extract the features for the validation set

In [11]:
X_validation = np.zeros((0,num_features))
y_validation = np.zeros((0))
for imgs, labels in val_loader:  # <3>
            outputs = model(imgs)  # <4>
            X_validation = np.concatenate((X_validation,outputs.detach().cpu().numpy().squeeze()))
            y_validation = np.concatenate((y_validation, labels))

In [12]:
X_validation.shape

(2000, 256)

In [13]:
y_train.shape

(10000,)

Train several classifiers with the extracted features

In [14]:
clf1 = GaussianNB()
clf1.fit(X_train, y_train)

clf2 = KNeighborsClassifier()
clf2.fit(X_train, y_train)

clf3 = DecisionTreeClassifier()
clf3.fit(X_train, y_train)

clf4 = QuadraticDiscriminantAnalysis()
clf4.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)

Measure the performance of the trained classifiers on the training and validation sets

In [15]:
print(clf1.score(X_train,y_train))
print(clf1.score(X_validation,y_validation))

0.8176
0.826


In [16]:
print(clf2.score(X_train,y_train))
print(clf2.score(X_validation,y_validation))

0.9033
0.8625


In [17]:
print(clf3.score(X_train,y_train))
print(clf3.score(X_validation,y_validation))

1.0
0.8085


In [18]:
print(clf4.score(X_train,y_train))
print(clf4.score(X_validation,y_validation))

0.8008
0.798
